In [1]:
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.utils.data
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler
from matplotlib import animation, rc
rc('animation', html='jshtml')
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [2]:
class Classifier(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Classifier,self).__init__()
        
        self.lstm1 = nn.LSTMCell(input_size, hidden_size)
        #self.lstm2 = nn.LSTMCell(hidden_size, hidden_size)
        self.layer1 = nn.Linear(hidden_size,hidden_size)
        self.ReLU1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size,hidden_size)
        self.ReLU2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_size,hidden_size)
        self.ReLU3 = nn.ReLU()
        self.layer4 = nn.Linear(hidden_size,output_size)
        self.Sigmoid = nn.Sigmoid()
    def forward(self,input):
        #print(input.shape)
        h_out = torch.zeros(input.size(0), 6).cuda()
        c_out = torch.zeros(input.size(0), 6).cuda()
        #h2_out = torch.zeros(input.size(0), 6).cuda()
        #c2_out = torch.zeros(input.size(0), 6).cuda()
        
        h_out, c_out = self.lstm1(input,(h_out, c_out))
        #h2_out, c2_out = self.lstm2(h_out,(h2_out, c2_out))
        #out = self.lstm2(out)
        out = self.layer1(h_out)
        out = self.ReLU1(out)
        #out = self.layer2(out)
        #out = self.ReLU2(out)
        #out = self.layer3(out)
        #out = self.ReLU3(out)
        out = self.layer4(out)
        out = self.Sigmoid(out)
        return out 

In [3]:
import pickle
with open('scaler.pickle','rb') as f:
    InpScaler = pickle.load(f)

In [4]:
model2 = Classifier(6,6,1)
model2.load_state_dict(torch.load('./ModelSave1.pt'))
model2.cuda()
LabelList = ['ts',*range(5)]
fileList = ["./SensorData/Sensor{}".format(k) for k in range(48)]
for file in fileList:
    print(file)
    df = pd.read_pickle(file)
    #Load into Torch Dataset. 
    Input = torch.tensor(InpScaler.transform(df[LabelList].values))
    #Output = torch.tensor(df['Labels'].values)

    MainDataSet = torch.utils.data.TensorDataset(Input)
    Main = torch.utils.data.DataLoader(MainDataSet,batch_size = 200000,num_workers = 1)
    full_results = torch.Tensor()
    #Run Model over the batches. 
    for inp in Main:
        inp = inp[0].cuda()
        output = model2.forward(inp.float())

        third_tensor = torch.cat((inp.cpu().float(),output.cpu().float()), 1)
        #third_tensor = torch.cat((third_tensor,Label.reshape(output.shape).float()), 1)
        full_results = torch.cat((full_results,third_tensor),0)
    df['Labels'] = full_results[:,-1].detach().numpy()
    df.to_pickle(file)


./SensorData/Sensor0
./SensorData/Sensor1
./SensorData/Sensor2
./SensorData/Sensor3
./SensorData/Sensor4
./SensorData/Sensor5
./SensorData/Sensor6
./SensorData/Sensor7
./SensorData/Sensor8
./SensorData/Sensor9
./SensorData/Sensor10
./SensorData/Sensor11
./SensorData/Sensor12
./SensorData/Sensor13
./SensorData/Sensor14
./SensorData/Sensor15
./SensorData/Sensor16
./SensorData/Sensor17
./SensorData/Sensor18
./SensorData/Sensor19
./SensorData/Sensor20
./SensorData/Sensor21
./SensorData/Sensor22
./SensorData/Sensor23
./SensorData/Sensor24
./SensorData/Sensor25
./SensorData/Sensor26
./SensorData/Sensor27
./SensorData/Sensor28
./SensorData/Sensor29
./SensorData/Sensor30
./SensorData/Sensor31
./SensorData/Sensor32
./SensorData/Sensor33
./SensorData/Sensor34
./SensorData/Sensor35
./SensorData/Sensor36
./SensorData/Sensor37
./SensorData/Sensor38
./SensorData/Sensor39
./SensorData/Sensor40
./SensorData/Sensor41
./SensorData/Sensor42
./SensorData/Sensor43
./SensorData/Sensor44
./SensorData/Sensor4

In [31]:
df


,time,ts,0,1,2,3,4,5,Labels
0,65.942857,0.174439,-0.061890,-0.021529,0.003130,0.013569,-0.006022,-0.000815,0.999677
16,65.943177,-0.042732,-0.061882,-0.021512,0.003130,0.013560,-0.006022,-0.000818,0.999715
32,65.943497,0.047534,-0.061858,-0.021462,0.003130,0.013532,-0.006022,-0.000827,0.999702
48,65.943817,0.026435,-0.061818,-0.021379,0.003130,0.013486,-0.006022,-0.000842,0.999706
64,65.944137,0.117016,-0.061761,-0.021263,0.003130,0.013421,-0.006022,-0.000864,0.999692
80,65.944457,-0.182361,-0.061689,-0.021116,0.003129,0.013339,-0.006023,-0.000891,0.999734
96,65.944777,-0.134272,-0.061599,-0.020938,0.003126,0.013239,-0.006025,-0.000924,0.999729
112,65.945097,-0.221756,-0.061491,-0.020732,0.003120,0.013123,-0.006028,-0.000964,0.999738
128,65.945417,0.045094,-0.061365,-0.020498,0.003112,0.012990,-0.006032,-0.001010,0.999708
144,65.945737,0.059056,-0.061223,-0.020238,0.003101,0.012842,-0.006038,-0.001062,0.999706


In [9]:
df.to_pickle




<bound method NDFrame.to_pickle of                time        ts         0         1         2         3  \
0         65.942857  0.174439 -0.061890 -0.021529  0.003130  0.013569   
16        65.943177 -0.042732 -0.061882 -0.021512  0.003130  0.013560   
32        65.943497  0.047534 -0.061858 -0.021462  0.003130  0.013532   
48        65.943817  0.026435 -0.061818 -0.021379  0.003130  0.013486   
64        65.944137  0.117016 -0.061761 -0.021263  0.003130  0.013421   
80        65.944457 -0.182361 -0.061689 -0.021116  0.003129  0.013339   
96        65.944777 -0.134272 -0.061599 -0.020938  0.003126  0.013239   
112       65.945097 -0.221756 -0.061491 -0.020732  0.003120  0.013123   
128       65.945417  0.045094 -0.061365 -0.020498  0.003112  0.012990   
144       65.945737  0.059056 -0.061223 -0.020238  0.003101  0.012842   
160       65.946057 -0.180248 -0.061064 -0.019954  0.003084  0.012680   
176       65.946377 -0.203546 -0.060890 -0.019647  0.003062  0.012503   
192       65.946